<a href="https://colab.research.google.com/github/pokjay/heb-squad/blob/main/preprocess_translate_squad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json

## Convert SQuAD JSON file to Pandas DataFrame

In [ ]:
with open('/content//dev-v2.0.json') as f_handle:
    dataset = json.load(f_handle)

In [ ]:
def handle_answer(qa, answer_field):

  rows = []

  for answer in qa[answer_field]:

    q = qa['question']
    qid = qa['id']
    is_impossible = qa['is_impossible']
    answer_end = answer['answer_start'] + len(answer['text'])
                
    context_marked = context[:answer['answer_start']] + \
                    '[0123] ' + \
                    context[answer['answer_start'] : answer_end] + \
                    ' [4567]' \
                    + context[answer_end:]
                
    row = {'article': article['title'],
            'id': qid,
            'context': context_marked,
            'question': q,
            'answer': answer['text'],
            'answer_start': answer['answer_start'],
            'answer_end': answer_end,
            'is_impossible': int(is_impossible)}

    row.append(row)
  
  return rows

In [ ]:
rows = []

for article in dataset['data']:
  for p in article['paragraphs']:
    # Some lines have \n, we remove them as they screw up translation
    context = p['context'].replace('\n', '')
    
    for qa in p['qas']:
      if 'answer' in qa:
        rows.extend(handle_answer(qa, 'answer'))
      if 'plausible_answers' in qa:
        rows.extend(handle_answer(qa, 'plausible_answers'))
      
